# Hamming-distance based error correction of mutant counts 

##### Adapted from JM_hamming_analysis by Thuy N. Nguyen on 191118

**Edited on 200928 for processing HiSeq data from 200914.**
**Edited on 210211 for processing subsampled Replicate 1 HiSeq data**
**Edited on 210219 for processing subsampled Q33S TYMS Hiseq data**

Purpose: To take mutant counts measured by NGS and correct these counts according to expected error of sequencing a pair of codons that are likely to be mis-assigned as another. TNN is revising JM's code to be more brief and less constrained to the parameters of the experiment. 

Some assumptions:
- The mutants sequenced are from a saturation mutagenesis generated by mutating each codon to all possible amino acids by mutageneic NNS primer using inverse PCR. 
- The FASTQ files returned from NGS are processed to merge overlapping paired-end reads using a program like USEARCH or FLASh. 
- These FASTQ files containing merged paired-end reads have been processed by CountingAlleles.py, which filters each read using a Quality Score threshold of 20 (1/100 error rate) and counts all mutants in the FASTQ file. This script will also count mutants that are in this FASTQ file. 
- The output of CountingAllles.py is a tab-delimited file containing the identity of the mutant and counts of a mutant codon. EX: '1ATG\t200'

In [37]:
import numpy as np
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna
from scipy import stats #as stats

In [38]:
def translate_seq(seq):
    #for going from nucleotides to aa
    seq = Seq(seq,generic_dna)
    seq_translate = str(seq.translate().strip())
    return seq_translate

def aa2codon(aminoAcidPosition,aaPositionList,wtAAsequence,wtDNASequence):
    #for a given amino acid position, this function will return the WT codon sequence 
    #See example above for how this function works if you need a clearer explanation
    
    aaPIX = list(aaPositionList).index(aminoAcidPosition)

    c1 = aaPIX*3 #codon positions in the WT DNA sequence
    c3 = c1+3
    
    WTcodon = wtDNASequence[c1:c3]
    #print(translate_seq(WTcodon))
    if translate_seq(WTcodon) == wtAAsequence[aaPIX]:
        return WTcodon
    else:
        return 'error'

def hammingDistance(ham_calc_wt,ham_calc_compare):
    ham_dist = 3
    for i in range(0, 3):
        if ham_calc_wt[i] == ham_calc_compare[i]:
            ham_dist = np.subtract(ham_dist, 1)
        else:
            continue
    return(ham_dist)

def hammingCalc(codon1,codon2,mutCount,hamming_value = -3.0 ): # hamming value is: Q-score/-10 = 30/-10 = -3
    hammingCountError = 0
    if codon1 != codon2:
        ham_dist = hammingDistance(codon1,codon2)
        hammingCountError = (mutCount*(np.power(np.power(10,hamming_value),ham_dist)))
    else: 
        hammingCountError = 0 #identical codons will not have this hamming problem 
        
    return hammingCountError 

### Initialize variables 
- path of your file of mutant counts

- WT sequence of your protein of interest (both DNA and amino acid)
- Dictionary that will be filled with your raw mutant counts
- Dictionary that will be filled with your corrected mutant counts

### Define paths for each sample and corresponding sample names.
- The data will be organized in a dictionary and will be keyed by this sample name. 

In [39]:
path = 'subsampled_allele_counts/'

outname = [i for i in range(1,73)]
pathMutCounts = []

for f in outname: 
    pathMutCounts.append(path+'countingAllelesOut_%snuc.txt'% f)

pathMutCounts

['subsampled_allele_counts/countingAllelesOut_1nuc.txt',
 'subsampled_allele_counts/countingAllelesOut_2nuc.txt',
 'subsampled_allele_counts/countingAllelesOut_3nuc.txt',
 'subsampled_allele_counts/countingAllelesOut_4nuc.txt',
 'subsampled_allele_counts/countingAllelesOut_5nuc.txt',
 'subsampled_allele_counts/countingAllelesOut_6nuc.txt',
 'subsampled_allele_counts/countingAllelesOut_7nuc.txt',
 'subsampled_allele_counts/countingAllelesOut_8nuc.txt',
 'subsampled_allele_counts/countingAllelesOut_9nuc.txt',
 'subsampled_allele_counts/countingAllelesOut_10nuc.txt',
 'subsampled_allele_counts/countingAllelesOut_11nuc.txt',
 'subsampled_allele_counts/countingAllelesOut_12nuc.txt',
 'subsampled_allele_counts/countingAllelesOut_13nuc.txt',
 'subsampled_allele_counts/countingAllelesOut_14nuc.txt',
 'subsampled_allele_counts/countingAllelesOut_15nuc.txt',
 'subsampled_allele_counts/countingAllelesOut_16nuc.txt',
 'subsampled_allele_counts/countingAllelesOut_17nuc.txt',
 'subsampled_allele_cou

In [40]:
#to build my list of all 64 codons. These are the amino acids in alphebetical order. 
# What are all possible codons that you can be mutated to? 

codon_list = ['GCT', 'GCC', 'GCA', 'GCG', 'CGT', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG', 'AAT', 'AAC', 'GAT', 'GAC', 'TGT', 'TGC'\
, 'CAA', 'CAG', 'GAA', 'GAG', 'GGT', 'GGC', 'GGA', 'GGG', 'CAT', 'CAC', 'ATT', 'ATC', 'ATA'\
, 'CTT', 'CTC', 'CTA', 'CTG', 'TTA', 'TTG', 'AAA', 'AAG', 'ATG', 'TTT', 'TTC', 'CCT', 'CCC', 'CCA', 'CCG'\
, 'TCT', 'TCC', 'TCA', 'TCG', 'AGT', 'AGC', 'ACT', 'ACC', 'ACA', 'ACG', 'TGG', 'TAT', 'TAC'\
, 'GTT', 'GTC', 'GTA', 'GTG', 'TAA', 'TAG', 'TGA']


# All possible amino acids that you can be mutated to
AA = 'ACDEFGHIKLMNPQRSTVWY*' 

In [41]:
# all possible amino acids mutants in DHFR, organized by sub-library 
sublibraries = ['SL1','SL2','SL3','SL4']

conditions = np.arange(1,27) #later initialize with actual sample IDs

seq_wt_aa = {}
seq_wt_aa['full'] = 'MISLIAALAVDRVIGMENAMPWNLPADLAWFKRNTLNKPVIMGRHTWESIGRPLPGRKNIILSSQPGTDDRVTWVKSVDEAIAACGDVPEIMVIGGGRVYEQFLPKAQKLYLTHIDAEVEGDTHFPDYEPDDWESVFSEFHDADAQNSHSYCFEILERR*'
seq_wt_aa['SL1']= 'MISLIAALAVDRVIGMENAMPWNLPADLAWFKRNTLNKPV'
seq_wt_aa['SL2']= 'IMGRHTWESIGRPLPGRKNIILSSQPGTDDRVTWVKSVDE'
seq_wt_aa['SL3']= 'AIAACGDVPEIMVIGGGRVYEQFLPKAQKLYLTHIDAEVE'
seq_wt_aa['SL4']= 'GDTHFPDYEPDDWESVFSEFHDADAQNSHSYCFEILERR*'

# Python index of amino acid position in DHFR 
aa_pos = {}
aa_pos['full'] = np.arange(1,160+1)
aa_pos['SL1'] = np.arange(1,40+1)  
aa_pos['SL2'] = np.arange(41,80+1)
aa_pos['SL3'] = np.arange(81,120+1)
aa_pos['SL4'] = np.arange(121,160+1)


# Do the same for DNA sequence
dna_pos = {}
for sl in sublibraries:
    dna_pos[sl] = []
    for aix in aa_pos[sl]:
        codon1 = aix*3-2
        codon2 = aix*3-1
        codon3 = aix*3
        dna_pos[sl].append(codon1)
        dna_pos[sl].append(codon2)
        dna_pos[sl].append(codon3)
        
seq_wt_dna = {}
seq_wt_dna['full'] = 'ATGATCAGTCTGATTGCGGCGTTAGCGGTAGATCGCGTTATCGGCATGGAAAACGCCATGCCGTGGAACCTGCCTGCCGATCTCGCCTGGTTTAAACGCAACACCTTAAATAAACCCGTGATTATGGGCCGCCATACCTGGGAATCAATCGGTCGTCCGTTGCCAGGACGCAAAAATATTATCCTCAGCAGTCAACCGGGTACGGACGATCGCGTAACGTGGGTGAAGTCGGTGGATGAAGCCATCGCGGCGTGTGGTGACGTACCAGAAATCATGGTGATTGGCGGCGGTCGCGTTTATGAACAGTTCTTGCCAAAAGCGCAAAAACTGTATCTGACGCATATCGACGCAGAAGTGGAAGGCGACACCCATTTCCCGGATTACGAGCCGGATGACTGGGAATCGGTATTCAGCGAATTCCACGATGCTGATGCGCAGAACTCTCACAGCTATTGCTTTGAGATTCTGGAGCGGCGGTAA'
for sl in sublibraries:
    seq_wt_dna[sl] = seq_wt_dna['full'][dna_pos[sl][0]-1:dna_pos[sl][-1]]

#check to see if DNA sequences for each SL is defined accurately 
tmp = seq_wt_dna['SL1'] + seq_wt_dna['SL2'] + seq_wt_dna['SL3'] + seq_wt_dna['SL4']
for aix, a in enumerate(tmp): 
    if a != seq_wt_dna['full'][aix]:
        #print aix, a, seq_wt_dna['full'][aix] #this will spit out mis-matched amino acid positions 
        print('incorrect DNA sequence')
for sl in sublibraries:
    if seq_wt_aa[sl] != translate_seq(seq_wt_dna[sl]):
        print('incorrect amino acid sequence')

Import mutant counts in each sample and organize into a dictionary that is structured like this: 
   
   * the structure of the data is sorted entirely into data_dict
   
            >> data_dict[name of sample][SL1 or SL2][Mut] = mutant count
            


In [42]:
#importing each sample and storing the data in the form of a dictionary

data_dict = {}
#import the counts
for sampleNameix, sampleName in enumerate(outname):
    file = open(pathMutCounts[sampleNameix],'r')
    data = file.readlines()
    
    data_dict[sampleName] = {}
    for line in data:
        
        sp_line = line.split('\t') 
        if sp_line[0][0:2] != 'sl':
            continue #skip over statistics in this file 
        
        else: 
            sl_id = sp_line[0].upper() #define sublibrary,
            mut = sp_line[1] # mutant name
            mut_count = float(sp_line[2].strip()) # and mutant count
            
            if sl_id not in data_dict[sampleName].keys():
                
                data_dict[sampleName][sl_id] = {}

            data_dict[sampleName][sl_id][mut] = mut_count


In [43]:
#This is to build matrix of all counts. 
#full_codon_matrix is the size, codon_list is the nucelotide list
nucleotide_matrix = {}
for condition in list(data_dict.keys()):
    reads_mat = np.zeros(shape = (len(aa_pos['full']),len(codon_list)))
    for slix,sl in enumerate(sublibraries):
        if sl in data_dict[condition].keys():
            for ipos, pos in enumerate(aa_pos[sl]):
                for icodon, codon in enumerate(codon_list):
                    wtCodon = aa2codon(pos,aa_pos[sl],seq_wt_aa[sl],seq_wt_dna[sl])
                    if codon[-1] == 'A':
                        continue
                    if codon[-1] == 'T':
                        continue
                    if wtCodon != codon:
                        mutCodon = str(pos)+codon
                        if mutCodon in data_dict[condition][sl].keys():
                            reads_mat[pos-1,icodon] = data_dict[condition][sl][mutCodon]
        nucleotide_matrix[condition] = reads_mat

In [44]:
# Description of your hamming filter here: 

#first, initialize dictionary with WT raw reads. As the thing iterates through each codon, 
#it will subtract error reads 
wtRawCounts = {}
wtHammingCounts = {} #also initialize a dictionray that will store your hamming WT reads that are error
wtHammingCorrected = {}
for cond in outname:
    wtRawCounts[cond] = {}
    wtHammingCounts[cond] = {} #also initialize a dictionray that will store your hamming WT reads that are error
    wtHammingCorrected[cond] = {}

    for slix, sl in enumerate(sublibraries):
        if sl in data_dict[cond].keys():
            wtRawCounts[cond][sl] = data_dict[cond][sl]['WT']
            wtHammingCounts[cond][sl] = 0
            wtHammingCorrected[cond][sl] = 0

for cond in outname: #37 of em. 
    for slix, sl in enumerate(sublibraries):
        if sl in data_dict[cond].keys():
            wtHammingCounts[cond][sl] = 0
            
            for ix, ipos in enumerate(aa_pos[sl]):
                for jx, refCodon in enumerate(codon_list):
                    wtCodon = aa2codon(ipos, aa_pos[sl],seq_wt_aa[sl],seq_wt_dna[sl])
                    if wtCodon == refCodon:
                        hammingCount = 0
                        
                        for kx, queryCodon in enumerate(codon_list):
                            nMut = nucleotide_matrix[cond][ipos-1,kx]
                            tmpHamCount = hammingCalc(refCodon,queryCodon,nMut)
                            hammingCount+=tmpHamCount
                        wtHammingCounts[cond][sl] = wtHammingCounts[cond][sl] + hammingCount

            #determine WT reads in each sub-library are due to hamming distance error. 
            wtHammingCorrected[cond][sl] = wtRawCounts[cond][sl] - wtHammingCounts[cond][sl]

In [45]:
mutHammingCount = {}
for condition in outname:
    mutHammingCount[condition] = np.zeros(shape = (len(aa_pos['full']),len(codon_list)))
    for ipos, pos in enumerate(aa_pos['full']):
        for jx, refCodon in enumerate(codon_list):
            hammingCounts = 0
            for kx, queryCodon in enumerate(codon_list):
                queryCodonMutCounts = nucleotide_matrix[condition][ipos,kx]
                tmpHamCount = hammingCalc(refCodon,queryCodon,queryCodonMutCounts)
                hammingCounts =+ tmpHamCount
            mutHammingCount[condition][ipos,jx] = hammingCounts

mutHammingCorrected= {}
for condition in outname:
    mutHammingCorrected[condition] = np.zeros(shape = (len(aa_pos['full']),len(codon_list)))
    for ipos, pos in enumerate(aa_pos['full']):
        for jx, refCodon in enumerate(codon_list):
            diff = nucleotide_matrix[condition][ipos,jx] - mutHammingCount[condition][ipos,jx]
            if diff <0:
                mutHammingCorrected[condition][ipos,jx] = 0
            else:
                mutHammingCorrected[condition][ipos,jx] = diff

In [46]:
## AMINO ACID MUTANT COUNTS #This needs to be done in separately after you have corrected DNA codon reads

#later loop through each condition
aa_countsTN = {}
for cond in outname:
    aa_countsTN[cond] = {}
    for sl in sublibraries:
        aa_countsTN[cond][sl] = {}

        for ipos, pos in enumerate(aa_pos[sl]):
            for icodon, codon in enumerate(codon_list):
                if seq_wt_aa[sl][ipos] == translate_seq(codon):
                    continue #skip WT
                else:
                    mutCount = mutHammingCorrected[cond][pos-1,icodon]
                    mutCodon = seq_wt_aa[sl][ipos]+str(pos)+translate_seq(codon)

                    if mutCodon in aa_countsTN[cond][sl].keys():
                        aa_countsTN[cond][sl][mutCodon] += mutCount
                    else: 
                        aa_countsTN[cond][sl][mutCodon] = mutCount

    for sl in sublibraries: #add WT
        if sl in wtHammingCorrected[cond].keys():
            aa_countsTN[cond][sl]['WT'] = wtHammingCorrected[cond][sl]

In [47]:
for cond in outname:
    output_file= open('hamOut/'+str(cond)+'.txt','w') #iterate through each condition later. 
    for sl in aa_countsTN[cond].keys():
        for m in aa_countsTN[cond][sl]:
            mCounts = str(aa_countsTN[cond][sl][m])
            output_file.write(sl+'\t'+m+'\t'+mCounts+'\n')
    output_file.close()